In [43]:
# Importações
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, regexp_extract, col, count, sum, when, countDistinct, initcap
from pyspark.sql.types import IntegerType, FloatType
from functools import reduce

In [3]:
# Criação do ambiente Spark
spark = SparkSession.builder \
.appName('Prata1') \
.getOrCreate()

In [4]:
# Carregamento do dataset inicial (df1)
df = spark.read.parquet(r"C:\Users\fred\Documents\Estudo de dados\Projeto\PetLovers\Dados\Sentimentos", header = True)
df.show()

+----+--------------------+----------+-------------+---------+
|  ID|         Comentarios|sentimento|Palavra_Chave|percepcao|
+----+--------------------+----------+-------------+---------+
|1419|Está muito caro e...|  negativo|         caro|   neutro|
|1419|Eu adoro este tap...|  positivo|         NULL|   neutro|
|1419|Absorve bem. Indico.|  positivo|       indico|   neutro|
|1419|Já usei vários ta...|  positivo|    higiênico| positivo|
|1419|Excelente qualida...|  positivo|    excelente| positivo|
|1419|O produto é excel...|  positivo|    excelente| positivo|
|1419|Estou sempre comp...|  positivo|        gosto| positivo|
|1419|Muito boa qualida...|  positivo|          bom| positivo|
|1419|O super secão é o...|  positivo|        secão|   neutro|
|1419|      Já foi melhor.|    neutro|          bom| positivo|
|1419|              Ótimo.|  negativo|          bom| positivo|
|1419|Meu cachorro so a...|  negativo|         NULL|   neutro|
|1419|Muito bom, não va...|  positivo|          bom| po

In [37]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Comentarios: string (nullable = true)
 |-- sentimento: string (nullable = true)
 |-- Palavra_Chave: string (nullable = true)
 |-- percepcao: string (nullable = true)



In [42]:
totais = df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df.columns
])

totais.show()

+---+-----------+----------+-------------+---------+
| ID|Comentarios|sentimento|Palavra_Chave|percepcao|
+---+-----------+----------+-------------+---------+
|  0|          0|         0|         9059|        0|
+---+-----------+----------+-------------+---------+



In [48]:
df = df.withColumn("sentimento", initcap("sentimento")) \
       .withColumn("Palavra_Chave", initcap("Palavra_Chave")) \
       .withColumn("percepcao", initcap("percepcao"))

In [51]:
df = df.withColumnRenamed('sentimento', 'Sentimento').withColumnRenamed('percepcao', 'Percepcao')
df.show()

+----+--------------------+----------+-------------+---------+
|  ID|         Comentarios|Sentimento|Palavra_Chave|Percepcao|
+----+--------------------+----------+-------------+---------+
|1419|Está muito caro e...|  Negativo|         Caro|   Neutro|
|1419|Eu adoro este tap...|  Positivo|         NULL|   Neutro|
|1419|Absorve bem. Indico.|  Positivo|       Indico|   Neutro|
|1419|Já usei vários ta...|  Positivo|    Higiênico| Positivo|
|1419|Excelente qualida...|  Positivo|    Excelente| Positivo|
|1419|O produto é excel...|  Positivo|    Excelente| Positivo|
|1419|Estou sempre comp...|  Positivo|        Gosto| Positivo|
|1419|Muito boa qualida...|  Positivo|          Bom| Positivo|
|1419|O super secão é o...|  Positivo|        Secão|   Neutro|
|1419|      Já foi melhor.|    Neutro|          Bom| Positivo|
|1419|              Ótimo.|  Negativo|          Bom| Positivo|
|1419|Meu cachorro so a...|  Negativo|         NULL|   Neutro|
|1419|Muito bom, não va...|  Positivo|          Bom| Po

In [53]:
df.write.parquet(r"C:\Users\fred\Documents\Estudo de dados\Projeto\PetLovers\Dados\sentimentos_tratado")

In [58]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Contagem de sentimentos por ID
sentimentos_count = (
    df.groupBy("ID", "Sentimento")
    .agg(F.count("*").alias("count"))
    .groupBy("ID")
    .pivot("Sentimento", ["Positivo", "Negativo", "Neutro"])
    .agg(F.first("count"))
    .withColumnRenamed("Positivo", "Qtd_Positivo")
    .withColumnRenamed("Negativo", "Qtd_Negativo")
    .withColumnRenamed("Neutro", "Qtd_Neutro")
)

# Palavra-chave mais comum por ID e sentimento
palavra_freq = (
    df.groupBy("ID", "Sentimento", "Palavra_Chave")
    .agg(F.count("*").alias("freq"))
)

window_spec = Window.partitionBy("ID", "Sentimento").orderBy(F.desc("freq"))

palavra_top = (
    palavra_freq.withColumn("rank", F.row_number().over(window_spec))
    .filter("rank = 1")
    .select("ID", "Sentimento", "Palavra_Chave")
    .groupBy("ID")
    .pivot("Sentimento", ["Positivo", "Negativo", "Neutro"])
    .agg(F.first("Palavra_Chave"))
    .withColumnRenamed("Positivo", "Chave_Positivo")
    .withColumnRenamed("Negativo", "Chave_Negativo")
    .withColumnRenamed("Neutro", "Chave_Neutro")
)

# Resultado final limpo e elegante
df_enriquecido = (
    sentimentos_count.join(palavra_top, on="ID", how="outer")
    .orderBy("ID")
)

df_enriquecido.show(truncate=False)

+----+------------+------------+----------+--------------+--------------+------------+
|ID  |Qtd_Positivo|Qtd_Negativo|Qtd_Neutro|Chave_Positivo|Chave_Negativo|Chave_Neutro|
+----+------------+------------+----------+--------------+--------------+------------+
|0   |6           |3           |2         |Confortável   |Excelente     |NULL        |
|1   |14          |5           |NULL      |NULL          |Otimo         |NULL        |
|10  |2           |NULL        |NULL      |NULL          |NULL          |NULL        |
|100 |NULL        |1           |NULL      |NULL          |NULL          |NULL        |
|1000|5           |NULL        |NULL      |Bom           |NULL          |NULL        |
|1001|16          |9           |2         |Bom           |NULL          |NULL        |
|1002|1           |NULL        |NULL      |Bom           |NULL          |NULL        |
|1003|1           |NULL        |NULL      |Bom           |NULL          |NULL        |
|1004|NULL        |1           |NULL      |

In [61]:
sem_nulos = df_enriquecido.withColumn("Qtd_Positivo", when(col("Qtd_Positivo").isNull(), 0).otherwise(col("Qtd_Positivo"))). \
                           withColumn("Qtd_Negativo", when(col("Qtd_Negativo").isNull(), 0).otherwise(col("Qtd_Negativo"))). \
                           withColumn("Qtd_Neutro", when(col("Qtd_Neutro").isNull(), 0).otherwise(col("Qtd_Neutro")))

In [62]:
sem_nulos.show()

+----+------------+------------+----------+--------------+--------------+------------+
|  ID|Qtd_Positivo|Qtd_Negativo|Qtd_Neutro|Chave_Positivo|Chave_Negativo|Chave_Neutro|
+----+------------+------------+----------+--------------+--------------+------------+
|   0|           6|           3|         2|   Confortável|     Excelente|        NULL|
|   1|          14|           5|         0|          NULL|         Otimo|        NULL|
|  10|           2|           0|         0|          NULL|          NULL|        NULL|
| 100|           0|           1|         0|          NULL|          NULL|        NULL|
|1000|           5|           0|         0|           Bom|          NULL|        NULL|
|1001|          16|           9|         2|           Bom|          NULL|        NULL|
|1002|           1|           0|         0|           Bom|          NULL|        NULL|
|1003|           1|           0|         0|           Bom|          NULL|        NULL|
|1004|           0|           1|         0|

In [64]:
dim_sentimentos.write.parquet(r"C:\Users\fred\Documents\Estudo de dados\Projeto\PetLovers\Dados\modelo_estrela\dim_sentimentos")